In [173]:
def get_dir_path(s: str):
    return "docs/keyword_analysis/all_%s.html" % s

proportions = dict(width=1600, height=1000)
xl_proportions = dict(width=2000, height=1400)

In [174]:
from requests import head
import pandas as pd
from pathlib import Path
import re

files = Path("metadata/keywords/optimized_v2").glob("*.csv")
df = []
for file in files:
    try:
        file_df = pd.read_csv(file)
        file_df["fname"] = file
        df.append(file_df)
    except:
        print(f"unable to read file {file}")

df = pd.concat(df)
df.source = df.source.apply(lambda x: x.split('.')[1])
df['project'] = df.apply(lambda row: f"{row['author']}/{row['repo']}/{row['version']}", axis=1)
df.head()

,quality_attribute,sentence,source,author,repo,version,id,keyword,matched_word,match_idx,filename,wiki,url,total_similar,target_keywords,target_matched_words,fname,project
0,Availability,"""""""; A candidate generator for entity linking ...",CODE_COMMENT,allenai,scispacy,v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,avail,available,94,scispacy/candidate_generation.py,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,1,['avail'],['available'],metadata\keywords\optimized_v2\allenai.scispac...,allenai/scispacy/v0.5.5
1,Availability,"""""""; Given a URL, look for the corresponding d...",CODE_COMMENT,allenai,scispacy,v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,down,download,93,scispacy/file_cache.py,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,1,['down'],['download'],metadata\keywords\optimized_v2\allenai.scispac...,allenai/scispacy/v0.5.5
2,Availability,"""""""; Given something that might be a URL (or m...",CODE_COMMENT,allenai,scispacy,v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,down,download,102,scispacy/file_cache.py,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,1,['down'],['download'],metadata\keywords\optimized_v2\allenai.scispac...,allenai/scispacy/v0.5.5
3,Availability,"# Download to temporary file, then copy to cac...",CODE_COMMENT,allenai,scispacy,v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,down,download,118,scispacy/file_cache.py,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,1,['down'],['download'],metadata\keywords\optimized_v2\allenai.scispac...,allenai/scispacy/v0.5.5
4,Availability,"# URL, so get it from the cache (downloading i...",CODE_COMMENT,allenai,scispacy,v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,down,downloading,33,scispacy/file_cache.py,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,1,['down'],['downloading'],metadata\keywords\optimized_v2\allenai.scispac...,allenai/scispacy/v0.5.5


In [185]:
df.shape

(871927, 18)

In [176]:
df.groupby(["quality_attribute", "sentence", "matched_word"]).first().reset_index().shape

(870616, 18)

In [177]:
df.groupby(["quality_attribute", "sentence", "keyword"]).first().reset_index().shape

(870615, 18)

In [178]:
df.groupby(["quality_attribute", "sentence"]).first().reset_index().shape

(870614, 18)

In [179]:
df.groupby(["quality_attribute", "sentence", "source"]).first().reset_index().shape

(871341, 18)

In [180]:
df.groupby(["quality_attribute", "sentence", "project"]).first().reset_index().shape

(871203, 18)

In [181]:
df.groupby(["quality_attribute", "sentence", "project", "source"]).first().reset_index().shape

(871927, 18)

In [182]:
df.groupby(["quality_attribute", "sentence", "source", "author", "repo", "version"]).first().reset_index().shape

(871927, 18)

In [183]:
aa = df.groupby(["quality_attribute", "sentence", "source", "author", "repo", "version"]).agg(fnames=("fname", lambda x: sorted(x.unique())), fname_count=("fname", lambda x: x.nunique()))
resulting = df.merge(aa, on=["quality_attribute", "sentence", "source", "author", "repo", "version"])
resulting.reset_index().sort_values(["fname_count"], ascending=False)

,index,quality_attribute,sentence,source,author,repo,version,id,keyword,matched_word,...,filename,wiki,url,total_similar,target_keywords,target_matched_words,fname,project,fnames,fname_count
0,0,Availability,"""""""; A candidate generator for entity linking ...",CODE_COMMENT,allenai,scispacy,v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,avail,available,...,scispacy/candidate_generation.py,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,1,['avail'],['available'],metadata\keywords\optimized_v2\allenai.scispac...,allenai/scispacy/v0.5.5,[metadata\keywords\optimized_v2\allenai.scispa...,1
581277,581277,Availability,Build failed on mac1015/cxx17.; Running on mac...,ISSUE_COMMENT,root-project,root,v6-32-06,https://github.com/root-project/root/pull/5643...,Error,Errors,...,NaN,https://root.cern,https://github.com/root-project/root/pull/5643...,3,"['Error', 'error']","['Errors', 'error']",metadata\keywords\optimized_v2\root-project.ro...,root-project/root/v6-32-06,[metadata\keywords\optimized_v2\root-project.r...,1
581279,581279,Availability,Build failed on mac1015/cxx17.; Running on mac...,ISSUE_COMMENT,root-project,root,v6-32-06,https://github.com/root-project/root/pull/5644...,Error,Errors,...,NaN,https://root.cern,https://github.com/root-project/root/pull/5644...,3,"['Error', 'down']","['Error', 'Errors', 'download-Release']",metadata\keywords\optimized_v2\root-project.ro...,root-project/root/v6-32-06,[metadata\keywords\optimized_v2\root-project.r...,1
581280,581280,Availability,Build failed on mac1015/cxx17.; Running on mac...,ISSUE_COMMENT,root-project,root,v6-32-06,https://github.com/root-project/root/pull/5651...,Error,Errors,...,NaN,https://root.cern,https://github.com/root-project/root/pull/5651...,3,"['Error', 'error']","['Errors', 'error']",metadata\keywords\optimized_v2\root-project.ro...,root-project/root/v6-32-06,[metadata\keywords\optimized_v2\root-project.r...,1
581281,581281,Availability,Build failed on mac1015/cxx17.; Running on mac...,ISSUE_COMMENT,root-project,root,v6-32-06,https://github.com/root-project/root/pull/5654...,Error,Errors,...,NaN,https://root.cern,https://github.com/root-project/root/pull/5654...,3,"['Error', 'down']","['Error', 'Errors', 'download-Release']",metadata\keywords\optimized_v2\root-project.ro...,root-project/root/v6-32-06,[metadata\keywords\optimized_v2\root-project.r...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290645,290645,Integrability,-set extrapolation (and; automatically sets up...,WIKI,psi4,psi4,v1.9.1,https://psicode.org/psi4manual/1.9.x/psithonin...,wrap,wrapper,...,psi4manual/1.9.x/psithoninput.html,https://psicode.org,https://psicode.org/psi4manual/1.9.x/psithonin...,1,['wrap'],['wrapper'],metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,[metadata\keywords\optimized_v2\psi4.psi4.v1.9...,1
290646,290646,Integrability,"-stage SCF calculation on short, equilibrium, ...",WIKI,psi4,psi4,v1.9.1,https://psicode.org/psi4manual/4.0b4/_modules/...,wrap,wrappers,...,psi4manual/4.0b4/_modules/wrappers.html,https://psicode.org,https://psicode.org/psi4manual/4.0b4/_modules/...,3,"['Wrap', 'wrap']","['Wrap', 'wrappers']",metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,[metadata\keywords\optimized_v2\psi4.psi4.v1.9...,1
290647,290647,Integrability,. # CSXError ceased to be used by v1.4. Class ...,WIKI,psi4,psi4,v1.9.1,https://psicode.org/psi4manual/1.8.x/_modules/...,message,message,...,psi4manual/1.8.x/_modules/psi4/driver/p4util/e...,https://psicode.org,https://psicode.org/psi4manual/1.8.x/_modules/...,6,['message'],['message'],metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,[metadata\keywords\optimized_v2\psi4.psi4.v1.9...,1
290648,290648,Integrability,". ## Force Python 3 print syntax, if this is p...",WIKI,psi4,psi4,v1.9.1,https://psicode.org/psi4manual/4.0b5/_modules/...,wrap,wrap,...,psi4manual/4.0b5/_modules/inputparser.html,https://

In [184]:
resulting.to_csv("metadata/results/duplicate_analysis_optimized.csv")

In [86]:
df.groupby(["sentence", "source", "project"])["id"].count().reset_index().sort_values(["sentence", "id"], ascending=False)

,sentence,source,project,id
299968,👍... but do we at least have a list somewhere ...,ISSUE_COMMENT,broadinstitute/cromwell/87,1
299967,👍 looks clear!. [![Approved with PullApprove](...,ISSUE_COMMENT,broadinstitute/cromwell/87,1
299966,👍 assuming Travis is happy. Re-assign back if/...,ISSUE_COMMENT,broadinstitute/cromwell/87,1
299965,"👍 I don't particularly like ""common"" since it'...",ISSUE_COMMENT,broadinstitute/cromwell/87,1
299964,🐍🧠 | ❓ | ❓ | ❓ | ❓ |. ✅ = works normally with ...,DOCS,allenai/scispacy/v0.5.5,1
...,...,...,...,...
4,"\t%esi, %edi; \ttestb\t$-1, %dil; \tsete\t%al;...",DOCS,root-project/root/v6-32-06,1
3,\t\t<#if readFilter?? && readFilter?size != 0>...,DOCS,broadinstitute/gatk/4.6.0.0,1
2,\t\t GNU LESSER GENERAL PUBLIC LICENSE; \t\t V...,DOCS,soedinglab/MMseqs2/15-6f452,1
1,"\t\t\t\t |\t dfh->add_transformation(""Bbs"", ""b...",ISSUE_COMMENT,psi4/psi4/v1.9.1,1


In [186]:
df.groupby(["sentence"]).value_counts(["project"])

sentence                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [189]:
df["in_project"] = df.groupby("sentence")["project"].transform(lambda x: x.nunique())
df.sort_values(["in_project", "sentence"], ascending=False)

,quality_attribute,sentence,source,author,repo,version,id,keyword,matched_word,match_idx,filename,wiki,url,total_similar,target_keywords,target_matched_words,fname,project,in_project
141,Energy Efficiency,\t\t\t\t\t\t\t\t <; \t\t\t\t\t\t\t\t\t\t\t\t <...,ISSUE_COMMENT,psi4,psi4,v1.9.1,https://github.com/psi4/psi4/pull/1101#issueco...,Energy,Energy,10659,NaN,https://psicode.org,https://github.com/psi4/psi4/pull/1101#issueco...,1,['Energy'],['Energy'],metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,1
142,Energy Efficiency,\t\t\t\t\t\t\t\t <; 1Ag -30.481683 2Ag -1.2944...,ISSUE_COMMENT,psi4,psi4,v1.9.1,https://github.com/psi4/psi4/pull/1101#issueco...,Energy,Energy,10721,NaN,https://psicode.org,https://github.com/psi4/psi4/pull/1101#issueco...,1,['Energy'],['Energy'],metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,1
143,Energy Efficiency,\t\t\t\t\t\t Running in d2h symmetry. Rotation...,ISSUE_COMMENT,psi4,psi4,v1.9.1,https://github.com/psi4/psi4/pull/1101#issueco...,Energy,Energy,3124,NaN,https://psicode.org,https://github.com/psi4/psi4/pull/1101#issueco...,1,['Energy'],['Energy'],metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,1
144,Energy Efficiency,\t\t\t\t\t ==> Iterations <==. Total Energy De...,ISSUE_COMMENT,psi4,psi4,v1.9.1,https://github.com/psi4/psi4/pull/1101#issueco...,Energy,Energy,9346,NaN,https://psicode.org,https://github.com/psi4/psi4/pull/1101#issueco...,1,['Energy'],['Energy'],metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,1
1561,Usability,"\t\t\t\t If no name is given, prints out list ...",ISSUE_COMMENT,root-project,root,v6-32-06,https://github.com/root-project/root/pull/1012...,undo,undo,2512,NaN,https://root.cern,https://github.com/root-project/root/pull/1012...,4,['undo'],['undo'],metadata\keywords\optimized_v2\root-project.ro...,root-project/root/v6-32-06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,Deployability,Update README.md,ISSUE,COMBINE-lab,salmon,v1.10.1,https://github.com/COMBINE-lab/salmon/pull/63:0,Update,Update,0,NaN,https://combine-lab.github.io/salmon,https://github.com/COMBINE-lab/salmon/pull/63,2,['Update'],['Update'],metadata\keywords\optimized_v2\COMBINE-lab.sal...,COMBINE-lab/salmon/v1.10.1,8
396,Deployability,Update README.md,ISSUE,hail-is,hail,0.2.133,https://github.com/hail-is/hail/pull/948:0,Update,Update,0,NaN,https://hail.is,https://github.com/hail-is/hail/pull/948,6,['Update'],['Update'],metadata\keywords\optimized_v2\hail-is.hail.0....,hail-is/hail/0.2.133,8
925,Deployability,Update README.md,ISSUE,psi4,psi4,v1.9.1,https://github.com/psi4/psi4/pull/1563:0,Update,Update,0,NaN,https://psicode.org,https://github.com/psi4/psi4/pull/1563,1,['Update'],['Update'],metadata\keywords\optimized_v2\psi4.psi4.v1.9....,psi4/psi4/v1.9.1,8
1032,Deployability,Update README.md,ISSUE,qupath,qupath,v0.5.1,https://github.com/qupath/qupath/pull/672:0,Update,Update,0,NaN,https://qupath.github.io,https://github.com/qupath/qupath/pull/672,2,['Update'],['Update'],metadata\keywords\optimized_v2\qupath.qupath.v...,qupath/qupath/v0.5.1,8


In [108]:
# duplicate sentences across projects

selection = df.groupby(["sentence"])["project"].nunique().reset_index().query("project > 1")["sentence"]
df[df["sentence"].isin(selection)].sort_values(["sentence", "project"], ascending=False)

# df[selection[selection["project"] > 1].index].sort_values

,id,quality_attribute,keyword,matched_word,match_idx,sentence,source,filename,author,repo,version,wiki,url,project,base_url
28,https://github.com/su2code/SU2/tree/v8.1.0/LIC...,Security,threat,threat,2971,you must give the recipients all the rights th...,DOCS,LICENSE.md,su2code,SU2,v8.1.0,https://su2code.github.io,https://github.com/su2code/SU2/tree/v8.1.0/LIC...,su2code/SU2/v8.1.0,https://github.com/su2code/SU2/tree/v8.1.0/LIC...
125,https://github.com/sofa-framework/sofa/tree/v2...,Security,threat,threat,2957,you must give the recipients all the rights th...,DOCS,LICENSE-LGPL.md,sofa-framework,sofa,v24.06.00,https://www.sofa-framework.org,https://github.com/sofa-framework/sofa/tree/v2...,sofa-framework/sofa/v24.06.00,https://github.com/sofa-framework/sofa/tree/v2...
176,https://github.com/sofa-framework/sofa/tree/v2...,Security,threat,threat,2957,you must give the recipients all the rights th...,DOCS,applications/plugins/ArticulatedSystemPlugin/L...,sofa-framework,sofa,v24.06.00,https://www.sofa-framework.org,https://github.com/sofa-framework/sofa/tree/v2...,sofa-framework/sofa/v24.06.00,https://github.com/sofa-framework/sofa/tree/v2...
770,https://github.com/root-project/root/tree/v6-3...,Security,threat,threat,5240,you must give the recipients all the rights th...,DOCS,interpreter/cling/LICENSE.TXT,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,https://github.com/root-project/root/tree/v6-3...
471,https://github.com/sofa-framework/sofa/tree/v2...,Availability,avail,available,36286,xed term (regardless of how the transaction is...,DOCS,Sofa/GUI/Qt/libQGLViewer/licence.txt,sofa-framework,sofa,v24.06.00,https://www.sofa-framework.org,https://github.com/sofa-framework/sofa/tree/v2...,sofa-framework/sofa/v24.06.00,https://github.com/sofa-framework/sofa/tree/v2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,https://github.com/qupath/qupath/tree/v0.5.1/q...,Integrability,interface,interfaces,2028,"""Source"" form shall mean the preferred form f...",DOCS,qupath-app/licenses/ikonli/LICENSE.txt,qupath,qupath,v0.5.1,https://qupath.github.io,https://github.com/qupath/qupath/tree/v0.5.1/q...,qupath/qupath/v0.5.1,https://github.com/qupath/qupath/tree/v0.5.1/q...
562,https://github.com/qupath/qupath/tree/v0.5.1/q...,Integrability,interface,interfaces,2576,"""Source"" form shall mean the preferred form f...",DOCS,qupath-app/licenses/JavaCPP/LICENSE.txt,qupath,qupath,v0.5.1,https://qupath.github.io,https://github.com/qupath/qupath/tree/v0.5.1/q...,qupath/qupath/v0.5.1,https://github.com/qupath/qupath/tree/v0.5.1/q...
616,https://github.com/qupath/qupath/tree/v0.5.1/q...,Integrability,interface,interfaces,2027,"""Source"" form shall mean the preferred form f...",DOCS,qupath-app/licenses/Picocli/LICENSE.txt,qupath,qupath,v0.5.1,https://qupath.github.io,https://github.com/qupath/qupath/tree/v0.5.1/q...,qupath/qupath/v0.5.1,https://github.com/qupath/qupath/tree/v0.5.1/q...
633,https://github.com/qupath/qupath/tree/v0.5.1/q...,Integrability,interface,interfaces,2027,"""Source"" form shall mean the preferred form f...",DOCS,qupath-app/licenses/SnakeYAML/LICENSE.txt,qupath,qupath,v0.5.1,https://qupath.github.io,https://github.com/qupath/qupath/tree/v0.5.1/q...,qupath/qupath/v0.5.1,https://github.com/qupath/qupath/tree/v0.5.1/q...


In [122]:
## duplicate sentences across sources

# df.groupby(["quality_attribute", "sentence", "project"])["source"].nunique()
df.groupby(["quality_attribute", "sentence", "project"]).agg(source=("source", lambda x: sorted(x.unique())), total=("source", lambda x: x.nunique()))
# selection = df.groupby(["quality_attribute", "sentence", "project"])["source"].nunique().reset_index().query("source > 1")["sentence"]
# df[df["sentence"].isin(selection)].sort_values(["sentence", "project"], ascending=False)

# df[selection[selection["project"] > 1].index].sort_values

source  \
quality_attribute sentence                                           project                                        
Availability      \t\t<#if readFilter?? && readFilter?size != 0>;... broadinstitute/gatk/4.6.0.0           [DOCS]   
                  \t1607\t1609\tCF\tT038\tUMLS:C0010674; 25763772... allenai/scispacy/v0.5.5               [DOCS]   
                  \t; Remove Temporary Files \tfalse; ```. And th... soedinglab/MMseqs2/15-6f452  [ISSUE_COMMENT]   
                   \t``%ptrs = <%ptr, %ptr + %stride, %ptr + 2 * ... root-project/root/v6-32-06            [DOCS]   
                   ![New box option for 3D histograms](ReverseAxi... root-project/root/v6-32-06            [DOCS]   
...                                                                                                           ...   
Usability         ﻿=====================================; CodeVie... root-project/root/v6-32-06            [DOCS]   
                  👍 I don't particularly like "common" since it's... broadinstitute/cromwell/87   [ISSUE_COMMENT]   
                  👍 assuming Travis is happy. Re-assign back if/w... broadinstitute/cromwell/87   [ISSUE_COMMENT]   
                  👍 looks clear!. [![Approved with PullApprove](h... broadinstitute/cromwell/87   [ISSUE_COMMENT]   
                  👍... but do we at least have a list somewhere (... broadinstitute/cromwell/87   [ISSUE_COMMENT]   

                                                                                                  total  
quality_attribute sentence                                           project                             
Availability      \t\t<#if readFilter?? && readFilter?size != 0>;... broadinstitute/gatk/4.6.0.0      1  
                  \t1607\t1609\tCF\tT038\tUMLS:C0010674; 25763772... allenai/scispacy/v0.5.5          1  
                  \t; Remove Temporary Files \tfalse; ```. And th... soedinglab/MMseqs2/15-6f452      1  
                   \t``%ptrs = <%ptr, %ptr + %stride, %ptr + 2 * ... root-project/root/v6-32-06       1  
                   ![New box option for 3D histograms](ReverseAxi... root-project/root/v6-32-06       1  
...                                                                                                 ...  
Usability         ﻿=====================================; CodeVie... root-project/root/v6-32-06       1  
                  👍 I don't particularly like "common" since it's... broadinstitute/cromwell/87       1  
                  👍 assuming Travis is happy. Re-assign back if/w... broadinstitute/cromwell/87       1  
                  👍 looks clear!. [![Approved with PullApprove](h... broadinstitute/cromwell/87       1  
                  👍... but do we at least have a list somewhere (... broadinstitute/cromwell/87       1  

[382266 rows x 2 columns]

In [48]:
sub = df.iloc[0:100].copy()
sub["total_similar"] = sub.groupby(["quality_attribute", "sentence"])["keyword"].transform("size")
sub.groupby(["quality_attribute", "sentence"]).first().reset_index().sort_values(["total_similar", "sentence"], ascending=False)

,quality_attribute,sentence,id,keyword,matched_word,match_idx,source,filename,author,repo,version,wiki,url,project,base_url,total_similar
30,Modifiability,"""""""; A spaCy pipe for detecting hyponyms using...",https://github.com/allenai/scispacy/tree/v0.5....,config,config,630,CODE_COMMENT,scispacy/hyponym_detector.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,4
62,Security,"""""""; Instructions for creating a release of th...",https://github.com/allenai/scispacy/tree/v0.5....,password,password,458,CODE_COMMENT,setup.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,3
72,Testability,# this is really just testing that we handle t...,https://github.com/allenai/scispacy/tree/v0.5....,test,testing,22,CODE_COMMENT,tests/custom_tests/test_custom_segmentation.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,2
34,Modifiability,# These lookbehinds are commented out because ...,https://github.com/allenai/scispacy/tree/v0.5....,variab,variable,55,CODE_COMMENT,scispacy/custom_tokenizer.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,2
46,Performance,"# Download to temporary file, then copy to cac...",https://github.com/allenai/scispacy/tree/v0.5....,cache,cache,43,CODE_COMMENT,scispacy/file_cache.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Availability,"""""""; A candidate generator for entity linking ...",https://github.com/allenai/scispacy/tree/v0.5....,avail,available,94,CODE_COMMENT,scispacy/candidate_generation.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,1
29,Modifiability,"""""""; A candidate generator for entity linking ...",https://github.com/allenai/scispacy/tree/v0.5....,config,configured,197,CODE_COMMENT,scispacy/candidate_generation.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,1
12,Deployability,""""""". Convert a umls release to a jsonl file of...",https://github.com/allenai/scispacy/tree/v0.5....,release,release,20,CODE_COMMENT,scripts/export_umls_json.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,1
11,Deployability,given id using the kb attribute of this class...,https://github.com/allenai/scispacy/tree/v0.5....,pipeline,pipeline,1816,CODE_COMMENT,scispacy/linking.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,allenai/scispacy/v0.5.5,https://github.com/allenai/scispacy/tree/v0.5....,1


In [68]:
# df = df.drop(columns=["total_similar"])

In [128]:
# sub = df.iloc[0:1000].copy()
sub = df
transformations = sub.groupby(["quality_attribute", "sentence", "source", "project"]).agg(
    total_similar=("id", "count"),
    target_keywords=("keyword", lambda x: sorted(x.unique())),
    target_matched_words=("matched_word", lambda x: sorted(x.unique())),
)
sub = sub.groupby(["quality_attribute", "sentence"]).first().reset_index()
sub = sub.merge(transformations, on=["quality_attribute", "sentence"])
sub.sort_values(["total_similar", "sentence"], ascending=False)

,quality_attribute,sentence,id,keyword,matched_word,match_idx,source,filename,author,repo,version,wiki,url,project,base_url,total_similar_id,total_similar,target_keywords,target_matched_words
352815,Testability,Get the template type names (if any) used to ...,https://www.sofa-framework.org/api/master/sofa...,log,logging,9298,WIKI,api/master/sofa/html/classsofa_1_1component_1_...,sofa-framework,sofa,v24.06.00,https://www.sofa-framework.org,https://www.sofa-framework.org/api/master/sofa...,sofa-framework/sofa/v24.06.00,https://www.sofa-framework.org/api/master/sofa...,3072,3072,[log],[logging]
156112,Integrability,aramsOfInterest); virtual RooAbsPdf*RooAbsPdf:...,https://root.cern/root/html528/RooAddModel.htm...,depend,dependentOverlaps,9522,WIKI,root/html528/RooAddModel.html,root-project,root,v6-32-06,https://root.cern,https://root.cern/root/html528/RooAddModel.html,root-project/root/v6-32-06,https://root.cern/root/html528/RooAddModel.html,2070,2070,[depend],"[dependentOverlaps, dependsOn, dependsOnValue]"
165644,Integrability,l*RooAbsReal::createPlotProjection(const RooAr...,https://root.cern/root/html528/PiecewiseInterp...,depend,dependentOverlaps,8567,WIKI,root/html528/PiecewiseInterpolation.html,root-project,root,v6-32-06,https://root.cern,https://root.cern/root/html528/PiecewiseInterp...,root-project/root/v6-32-06,https://root.cern/root/html528/PiecewiseInterp...,1938,1938,[depend],"[dependentOverlaps, dependsOn, dependsOnValue]"
172564,Integrability,rgSet& paramsOfInterest); virtual RooAbsPdf*Ro...,https://root.cern/root/html526/Roo2DKeysPdf.ht...,depend,dependentOverlaps,8196,WIKI,root/html526/Roo2DKeysPdf.html,root-project,root,v6-32-06,https://root.cern,https://root.cern/root/html526/Roo2DKeysPdf.html,root-project/root/v6-32-06,https://root.cern/root/html526/Roo2DKeysPdf.html,714,714,[depend],"[dependentOverlaps, dependsOn, dependsOnValue]"
335590,Testability,aramsOfInterest); virtual RooAbsPdf*RooAbsPdf:...,https://root.cern/root/html528/RooAddModel.htm...,test,testArg,9581,WIKI,root/html528/RooAddModel.html,root-project,root,v6-32-06,https://root.cern,https://root.cern/root/html528/RooAddModel.html,root-project/root/v6-32-06,https://root.cern/root/html528/RooAddModel.html,690,690,[test],[testArg]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249715,Performance,"\t%xmm2, %xmm2, %xmm3; - - - 1.00 - 1.00 - - -...",https://github.com/root-project/root/tree/v6-3...,throughput,throughput,16334,DOCS,interpreter/llvm-project/llvm/docs/CommandGuid...,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,https://github.com/root-project/root/tree/v6-3...,1,1,[throughput],[throughput]
249714,Performance,"\t%esi, %edi; \ttestb\t$-1, %dil; \tsete\t%al;...",https://github.com/root-project/root/tree/v6-3...,load,load,37276,DOCS,interpreter/llvm-project/llvm/lib/Target/X86/R...,root-project,root,v6-32-06,https://root.cern,https://github.com/root-project/root/tree/v6-3...,root-project/root/v6-32-06,https://github.com/root-project/root/tree/v6-3...,1,1,[load],[load]
0,Availability,\t\t<#if readFilter?? && readFilter?size != 0>...,https://github.com/broadinstitute/gatk/tree/4....,avail,available,4312,DOCS,src/main/resources/org/broadinstitute/hellbend...,broadinstitute,gatk,4.6.0.0,https://software.broadinstitute.org/gatk,https://github.com/broadinstitute/gatk/tree/4....,broadinstitute/gatk/4.6.0.0,https://github.com/broadinstitute/gatk/tree/4....,1,1,[avail],[available]
52671,Deployability,\t\t GNU LESSER GENERAL PUBLIC LICENSE; \t\t V...,https://github.com/soedinglab/MMseqs2/tree/15-...,release,released,321,DOCS,lib/omptl/License.txt,soedinglab,MMseqs2,15-6f452,https://github.com/soedinglab/mmseqs2/wiki,https://github.com/soedinglab/MMseqs2/tree/15-...,soedinglab/MMseqs2/15-6f452,https://github.com/soedinglab/MMseqs2/tree/15-...,1,1,[release],[released]


In [129]:
sub["total_similar"].sum()

(894948, 894948)

In [ ]:
attribs_per_url = df.pivot_table(values="matched_word", index="url", columns="quality_attribute", aggfunc="size", fill_value=0)
attribs_per_url["total"] = attribs_per_url.sum(axis=1)
attribs_per_url = attribs_per_url.sort_values(["total", "url"], ascending=False)
attribs_per_url.to_csv("metadata/results/keywords/attribs_per_url.csv", index=False)
attribs_per_url.style.background_gradient(cmap="YlGnBu", axis=None)


In [ ]:
attribs_per_url = df.sort_values(["matched_word"], ascending=True).pivot_table(values="matched_word", index="url", columns="quality_attribute", aggfunc=list, fill_value="")
attribs_per_url["total"] = attribs_per_url.map(len).sum(axis=1)
attribs_per_url = attribs_per_url.sort_values(["total", "url"], ascending=False)
attribs_per_url.to_csv("metadata/results/keywords/attribs_per_url_words.csv", index=False)
attribs_per_url

In [ ]:
summary_df = df.groupby(["quality_attribute", "project"]).size().reset_index(name="count")
summary_df = summary_df.sort_values(["project", "count"], ascending=False)
summary_df.to_csv("metadata/repo_info/summary_by_project_and_quality_attribute.csv", index=False)
summary_df

In [ ]:
summary_df["count"].sum()

In [ ]:
summary_df = df.groupby(["project"]).size().reset_index(name="count")
summary_df = summary_df.sort_values(["count"], ascending=False)
summary_df.to_csv("metadata/repo_info/summary_by_project.csv", index=False)
summary_df

In [ ]:
summary_df = df.groupby(["quality_attribute"]).size().reset_index(name="count")
summary_df = summary_df.sort_values(["count"], ascending=False)
summary_df.to_csv("metadata/repo_info/summary_by_quality_attribute.csv", index=False)
summary_df

In [ ]:
summary_df = df.groupby(["quality_attribute"]).size().reset_index(name="count")
summary_df = summary_df.sort_values(["count"], ascending=False)
summary_df['count'].sum()

In [ ]:
import plotly.express as px

# Group by 'Quality Attribute' and 'source', counting keyword matches
matrix_df = df.groupby(['quality_attribute', 'source', 'project']).size().reset_index(name='count')

# Plot grouped bar chart
fig = px.bar(matrix_df, x='quality_attribute', y='count', color='source', barmode='group',
             labels={'quality_attribute': 'Quality Attribute', 'count': 'Keyword Matches'},
             animation_frame="project",
             title='Source vs. Quality Attribute Matrix')

fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("source_vs_quality_attr_matrix"))


In [ ]:
scatted_df = df
scatted_df['sentence_length'] = df['sentence'].str.len()
scatted_df = scatted_df.groupby(['quality_attribute', 'source', 'matched_word']).agg(count=('keyword', 'size'), avg_sentence_length=('sentence_length', 'mean')).reset_index()
scatted_df.head()
scat = px.scatter(scatted_df, x='avg_sentence_length', y='matched_word', color='source', size='count', title='Scatter Plot of Quality Attributes by Source')
scat.update_traces(marker=dict(sizemode='area', sizeref=1, sizemin=1))
scat.update_layout(xaxis=dict(tickangle=45), legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1.02), **proportions)
scat.show()

In [ ]:
fig = px.histogram(df, y='keyword', color='source', animation_frame="project", title='Histogram of Keyword Matches by Source')
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(width=1600, height=2000)
fig.show()

fig.write_html(get_dir_path("histogram_of_keyword_matches_by_source_animated"))

In [ ]:
fig = px.histogram(df, y='keyword', color='source', title='Histogram of Keyword Matches by Source')
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(width=1600, height=2000)
fig.show()

fig.write_html(get_dir_path("histogram_of_keyword_matches_by_source"))

In [ ]:
fig = px.histogram(df, y='keyword', color='quality_attribute', title='Histogram of Keyword Matches by Source')
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(width=1600, height=2000)
fig.show()

fig.write_html(get_dir_path("histogram_of_keyword_matches_by_quality_attribute"))

In [ ]:
# Group and count by source and quality attribute
stacked_df = df.groupby(['source', 'quality_attribute']).size().reset_index(name='count')

# Plot stacked bar chart
fig = px.bar(stacked_df, x='quality_attribute', y='count', color='source',
             labels={'source': 'source', 'count': 'Keyword Matches'},
             title='Stacked Bar Chart of Quality Attributes by Source')
fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("stacked_bar_chart_quality_attr_to_sources"))


In [ ]:
# Count keywords within each quality attribute
treemap_df = df.groupby(['quality_attribute', 'project', 'keyword', 'matched_word']).size().reset_index(name='count')

# Plot treemap
fig = px.treemap(treemap_df, path=['project', 'quality_attribute', 'keyword', 'matched_word'], values='count', color="quality_attribute", color_discrete_sequence=px.colors.qualitative.Pastel,
                 title='Matched Word Frequency per Quality Attribute Treemap')

fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("treemap_quality_attr_to_matched_words"))


In [ ]:
# Count keywords within each quality attribute
treemap_df = df.groupby(['quality_attribute', 'project', 'keyword', 'matched_word']).size().reset_index(name='count')

# Plot treemap
fig = px.treemap(treemap_df, path=['quality_attribute', 'project', 'keyword', 'matched_word'], values='count', color="project", color_discrete_sequence=px.colors.qualitative.Pastel,
                 title='Matched Word Frequency per Quality Attribute Treemap')

fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("treemap_quality_attr_to_matched_words_by_project"))


In [ ]:
# Count keywords within each quality attribute
treemap_df = df.groupby(['quality_attribute', 'source', "project", 'keyword', 'matched_word']).size().reset_index(name='count')

# Plot treemap
fig = px.treemap(treemap_df, path=['source', 'quality_attribute', 'project', 'keyword', 'matched_word'], values='count', color='quality_attribute', color_discrete_sequence=px.colors.qualitative.Pastel,
                 title='Matched Word Frequency per Quality Attribute Treemap')

fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("treemap_quality_attr_to_matched_words_with_source"))


In [ ]:
# Count keywords within each quality attribute
treemap_df = df.groupby(['quality_attribute', 'project', 'source', 'keyword', 'matched_word', 'sentence']).size().reset_index(name='count')

# Plot treemap
fig = px.treemap(treemap_df, path=['quality_attribute', 'project', 'source', 'keyword', 'matched_word', 'sentence'], values='count', color='source', color_discrete_sequence=px.colors.qualitative.Pastel,
                 title='Matched Word Frequency per Quality Attribute Treemap')

fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("treemap_quality_attr_to_matched_words_with_source_and_sentences"))


In [ ]:
radar_df = df.groupby(['quality_attribute', 'project']).size().reset_index(name="count")
radar_df.head(15)

In [ ]:
fig = px.line_polar(radar_df, r="count", theta='quality_attribute', animation_frame="project", line_close=True,
                    title='Quality Attribute Radar Chart by Source', markers=True)

fig.update_traces(fill='toself')
fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("radar_chart_quality_attr"))

In [ ]:
import networkx as nx
import plotly.graph_objects as go

# Create edges between keywords and quality attributes
edges = df[['keyword', 'quality_attribute']].drop_duplicates()
G = nx.from_pandas_edgelist(edges, 'keyword', 'quality_attribute')

# Plot with Plotly
pos = nx.spring_layout(G)
edge_trace = go.Scatter(x=[], y=[], line=dict(width=1, color='#888'), hoverinfo='none', mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

node_trace = go.Scatter(x=[], y=[], text=[], mode='markers+text', hoverinfo='text',
                        marker=dict(showscale=True, color=[], size=10, colorbar=dict(thickness=15)))

for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += (x,)
    node_trace['y'] += (y,)
    node_trace['text'] += (node,)
    node_trace['marker']['color'] += (1 if node in edges['quality_attribute'].values else 0,)

fig = go.Figure(data=[edge_trace, node_trace], layout=go.Layout(title='Keyword and Quality Attribute Network Graph'))
fig.update_layout(**proportions)
fig.show()


In [ ]:
# Count keywords per quality attribute per source
hist_df = df.groupby(['quality_attribute', 'source']).size().reset_index(name='count')

# Plot histogram
fig = px.histogram(hist_df, x='quality_attribute', y='count', color='source',
                   barmode='group', title='Comparative Histogram of Keyword Matches by Source')

fig.update_layout(**proportions)
fig.show()

fig.write_html(get_dir_path("comparative_histogram_quality_attr_to_sources"))


In [ ]:
sunburst_df = df.groupby(['quality_attribute', 'source', 'keyword']).size().reset_index(name='count')
sunburst_df.head()


In [ ]:
# Group data by quality attribute, source, and repository
sunburst_df = df.groupby(['quality_attribute', 'source', 'keyword']).size().reset_index(name='count')

# Plot sunburst
import plotly.express as px

fig = px.sunburst(sunburst_df, path=['quality_attribute', 'source', 'keyword'], values='count',
                  title='Parallel Sets Diagram of Quality Attributes, Sources, and Keywords')
fig.update_layout(**proportions)
fig.show()
fig.write_html(get_dir_path("parallel_sets_diagram_quality_attr_to_keywords"))


In [ ]:
# Group data by quality attribute, source, and repository
sunburst_df = df.groupby(['quality_attribute', 'source', 'matched_word']).size().reset_index(name='count')

# Plot sunburst
import plotly.express as px

fig = px.sunburst(sunburst_df, path=['quality_attribute', 'source', 'matched_word'], values='count',
                  title='Parallel Sets Diagram of Quality Attributes, Sources, and Matched words')
fig.update_layout(**proportions)
fig.show()
fig.write_html(get_dir_path("parallel_sets_diagram_quality_attr_to_matched_words"))


In [ ]:
# Group data by quality attribute, source, and repository
import re
sunburst_df = df.groupby(['keyword', "base_url", 'quality_attribute', 'source', 'project']).size().reset_index(name='count')
sunburst_df.sort_values(['keyword', 'source'])

In [ ]:
sunburst_df = sunburst_df.groupby(['keyword', 'quality_attribute', 'source', 'project']).size().reset_index(name='count')
sunburst_df.sort_values(['keyword', 'source'])

In [ ]:
fig = px.sunburst(sunburst_df, path=['quality_attribute', 'source', 'keyword', 'project'], values='count',
                  title='Parallel Sets Diagram of Quality Attributes, Sources, and Matched words')
fig.update_layout(**proportions)
fig.show()
fig.write_html(get_dir_path("parallel_sets_diagram_quality_attr_to_matched_words"))

In [ ]:
# Group data by quality attribute, source, and repository
sunburst_df = df.groupby(['quality_attribute', 'source', 'keyword', 'matched_word']).size().reset_index(name='count')

# Plot sunburst
import plotly.express as px

fig = px.sunburst(sunburst_df, path=['quality_attribute', 'source', 'keyword', 'matched_word'], values='count',
                  title='Parallel Sets Diagram of Quality Attributes, Sources, and Matched words')
fig.update_layout(**proportions)
fig.show()
fig.write_html(get_dir_path("parallel_sets_diagram_quality_attr_to_keywords_with_matched_words"))


In [ ]:
sankey_df = df.groupby(['quality_attribute', 'source', 'keyword']).size().reset_index(name='count')

# Create a list of unique nodes (quality attributes, sources, keywords)
nodes = pd.concat([sankey_df['quality_attribute'], sankey_df['source'], sankey_df['keyword']]).unique()
node_dict = {node: i for i, node in enumerate(nodes)}  # Map each node to a unique index

# Define source and target nodes for the Sankey diagram based on the groupings
sankey_df['quality_attr_index'] = sankey_df['quality_attribute'].map(node_dict)
sankey_df['source_index'] = sankey_df['source'].map(node_dict)
sankey_df['keyword_index'] = sankey_df['keyword'].map(node_dict)

# Define links from quality_attribute -> source and source -> keyword
source_target_links = (
    pd.concat([
        sankey_df[['quality_attr_index', 'source_index', 'count']].rename(columns={'quality_attr_index': 'source', 'source_index': 'target'}),
        sankey_df[['source_index', 'keyword_index', 'count']].rename(columns={'source_index': 'source', 'keyword_index': 'target'})
    ])
)

# Extract source, target, and count lists for the Sankey plot
sources = source_target_links['source'].tolist()
targets = source_target_links['target'].tolist()
counts = source_target_links['count'].tolist()

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes  # Use unique nodes as labels
    ),
    link=dict(
        source=sources,  # Indices of source nodes
        target=targets,  # Indices of target nodes
        value=counts     # Counts as link values
    )
)])

fig.update_layout(title_text="Quality Attributes, Sources, and Keywords Sankey Diagram", font_size=14, **proportions)
fig.show()

fig.write_html(get_dir_path("sankey_quality_attr_to_sources_and_keywords"))

In [ ]:
sankey_df = df.groupby(['quality_attribute', 'source', 'matched_word']).size().reset_index(name='count')

# Create a list of unique nodes (quality attributes, sources, keywords)
nodes = pd.concat([sankey_df['quality_attribute'], sankey_df['source'], sankey_df['matched_word']]).unique()
node_dict = {node: i for i, node in enumerate(nodes)}  # Map each node to a unique index

# Define source and target nodes for the Sankey diagram based on the groupings
sankey_df['quality_attr_index'] = sankey_df['quality_attribute'].map(node_dict)
sankey_df['source_index'] = sankey_df['source'].map(node_dict)
sankey_df['keyword_index'] = sankey_df['matched_word'].map(node_dict)

# Define links from quality_attribute -> source and source -> keyword
source_target_links = (
    pd.concat([
        sankey_df[['quality_attr_index', 'source_index', 'count']].rename(columns={'quality_attr_index': 'source', 'source_index': 'target'}),
        sankey_df[['source_index', 'keyword_index', 'count']].rename(columns={'source_index': 'source', 'keyword_index': 'target'})
    ])
)

# Extract source, target, and count lists for the Sankey plot
sources = source_target_links['source'].tolist()
targets = source_target_links['target'].tolist()
counts = source_target_links['count'].tolist()

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes  # Use unique nodes as labels
    ),
    link=dict(
        source=sources,  # Indices of source nodes
        target=targets,  # Indices of target nodes
        value=counts     # Counts as link values
    )
)])

fig.update_layout(title_text="Quality Attributes, Sources, and Keywords Sankey Diagram", font_size=14, **xl_proportions)
fig.show()

fig.write_html(get_dir_path("sankey_quality_attr_to_sources_and_matched_words"))

In [ ]:
# Group data by quality attribute and source, summing the counts
sankey_df = df.groupby(['quality_attribute', 'source']).size().reset_index(name='count')

# Create a list of unique nodes (quality attributes and sources only)
nodes = pd.concat([sankey_df['quality_attribute'], sankey_df['source']]).unique()
node_dict = {node: i for i, node in enumerate(nodes)}  # Map each node to a unique index

# Define source and target nodes for the Sankey diagram
sankey_df['quality_attr_index'] = sankey_df['quality_attribute'].map(node_dict)
sankey_df['source_index'] = sankey_df['source'].map(node_dict)

# Extract source, target, and count lists for the Sankey plot
sources = sankey_df['quality_attr_index'].tolist()
targets = sankey_df['source_index'].tolist()
counts = sankey_df['count'].tolist()

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes  # Use unique nodes as labels
    ),
    link=dict(
        source=sources,  # Indices of source nodes
        target=targets,  # Indices of target nodes
        value=counts     # Counts as link values
    )
)])

fig.update_layout(title_text="Quality Attributes to Sources Sankey Diagram", font_size=14, **proportions)
fig.show()

fig.write_html(get_dir_path("sankey_quality_attr_to_sources"))

In [ ]:
# Sort versions and aggregate data by version, source, and quality attribute
trend_df = df.groupby(['version', 'source', 'quality_attribute']).size().reset_index(name='count')
trend_df = trend_df.sort_values(by='version')

# Plot stacked area chart
fig = px.area(trend_df, x='version', y='count', color='quality_attribute', line_group='source',
              title='Stacked Area Chart of Quality Attribute Trends by Source')
fig.show()

